In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_master_exceedance_jf = pd.read_excel(r'C:\Users\pasindu.s\Desktop\Work-Scripts\FRA work\FEA Tool_MV31_Master_JF 1.xlsm', engine = "openpyxl",sheet_name="Exceedance Log")
df_master_receiver_jf = pd.read_excel(r'C:\Users\pasindu.s\Desktop\Work-Scripts\FRA work\FEA Tool_MV31_Master_JF 1.xlsm', engine = "openpyxl",sheet_name="Receiver Points")
df_master_exceedance_pf = pd.read_excel(r'C:\Users\pasindu.s\Desktop\Work-Scripts\FRA work\FEA Tool_MV31_Master_PF.xlsm', engine = "openpyxl", sheet_name = "Exceedance Log")
df_master_receiver_pf = pd.read_excel(r'C:\Users\pasindu.s\Desktop\Work-Scripts\FRA work\FEA Tool_MV31_Master_PF.xlsm', engine = "openpyxl", sheet_name = "Receiver Points")

In [3]:
df_master_exceedance_jf.head()

,Receiver ID,Hazard ID,Consequence Type,Hazard Location,Frequency,Distance,Consequence Time,Hazard,Unnamed: 8,Unnamed: 9,Receiver,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,y,z,x,y,z
1,Pri Str 3W-1,G01_A_01_3W_01_010_IFBF,JF,G01_A_01_3W_01,1.538329e-07,9.462578,60.0,196.9512,31.14552,0,188.125042,34.557164,0
2,Pri Str 3W-2,G01_A_01_3W_01_010_IFBF,JF,G01_A_01_3W_01,1.538329e-07,5.067334,60.0,196.9512,31.14552,0,193.204394,34.557164,0
3,Pri Str 3W-3,G01_A_01_3W_01_010_IFBF,JF,G01_A_01_3W_01,1.538329e-07,3.689278,60.0,196.9512,31.14552,0,198.355286,34.557164,0
4,Pri Str 3W-4,G01_A_01_3W_01_010_IFBF,JF,G01_A_01_3W_01,1.538329e-07,7.389658,60.0,196.9512,31.14552,0,203.506178,34.557164,0


In [4]:
df_master_exceedance_pf.head()

,Receiver ID,Hazard ID,Consequence Type,Hazard Location,Frequency,Distance,Consequence Time,Hazard,Unnamed: 8,Unnamed: 9,Receiver,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,y,z,x,y,z
1,Pri Str 3W-1,P01_A_01_3W_01_050_ISBF,PF,P01_A_01_3W_01,0.000006,11.815588,15.0,199.421,31.09183,0,188.125,34.55716,0
2,Pri Str 3W-2,P01_A_01_3W_01_050_ISBF,PF,P01_A_01_3W_01,0.000006,7.117206,15.0,199.421,31.09183,0,193.2044,34.55716,0
3,Pri Str 3W-3,P01_A_01_3W_01_050_ISBF,PF,P01_A_01_3W_01,0.000006,3.625497,15.0,199.421,31.09183,0,198.3553,34.55716,0
4,Pri Str 3W-4,P01_A_01_3W_01_050_ISBF,PF,P01_A_01_3W_01,0.000006,5.356993,15.0,199.421,31.09183,0,203.5062,34.55716,0


In [7]:
#adjusting columns
header_2 = df_master_exceedance_jf.iloc[0,:].tolist()
columns = ['Receiver ID','Hazard ID','Consequence Type','Hazard Location','Frequency','Distance','Consequence Time','Hazard','Hazard','Hazard','Receiver','Receiver','Receiver']
columns_combined = list()
for head,head2 in zip(header_2,columns):
    if pd.isna(head):
        columns_combined.append(head2)
    else:
        columns_combined.append("_".join([head2,head]))

In [8]:
df_master_exceedance_jf.columns = columns_combined
df_master_exceedance_jf.drop(index=[0],axis = 0,inplace = True)
df_master_exceedance_pf.columns = columns_combined
df_master_exceedance_pf.drop(index=[0],axis = 0,inplace = True)

In [13]:
df_master_receiver_jf.columns = df_master_receiver_jf.iloc[0,:].tolist()
df_master_receiver_jf.drop(index = [0], axis = 0, inplace = True)
df_master_receiver_pf.columns = df_master_receiver_pf.iloc[0,:].tolist()
df_master_receiver_pf.drop(index = [0], axis = 0, inplace = True)

In [50]:
unique_receivers_pf = df_master_receiver_pf["Receiver ID"].unique().tolist()
unique_receivers_jf = df_master_receiver_jf["Receiver ID"].unique().tolist()

In [51]:
required_columns = ['Receiver ID','Hazard ID','Consequence Type','Hazard Location','Frequency','Distance','Consequence Time']
jf_exceedance = None
pf_exceedance = None
for receiver in unique_receivers_jf:
    temp_jf = df_master_exceedance_jf.loc[df_master_exceedance_jf["Receiver ID"] == receiver].sort_values(by=["Receiver ID","Consequence Time"])[required_columns]
    if jf_exceedance is None:
        jf_exceedance = temp_jf
    else:
        jf_exceedance = jf_exceedance.append(temp_jf, ignore_index = False)

for receiver in unique_receivers_pf:
    temp_pf = df_master_exceedance_pf.loc[df_master_exceedance_pf["Receiver ID"] == receiver].sort_values(by=["Receiver ID","Consequence Time"])[required_columns]
    if pf_exceedance is None:
        pf_exceedance = temp_pf
    else:
        pf_exceedance = pf_exceedance.append(temp_pf, ignore_index = False)

In [52]:
jf_exceedance.reset_index(drop = True, inplace = True)
pf_exceedance.reset_index(drop = True, inplace = True)

In [53]:
jf_exceedance

,Receiver ID,Hazard ID,Consequence Type,Hazard Location,Frequency,Distance,Consequence Time
0,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,JF,S15_A_03_1S_01,1.451586e-09,50.970467,0.0
1,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,JF,S15_A_03_1S_01,1.451586e-09,51.697512,0.0
2,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,JF,S15_A_03_1S_01,1.451586e-09,52.431609,0.0
3,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,JF,S15_A_03_1S_01,1.451586e-09,52.664498,0.0
4,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,JF,S15_A_03_1S_01,1.451586e-09,52.173652,0.0
...,...,...,...,...,...,...,...
557751,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,JF,S49_A_01_6W_01,2.152915e-08,6.966048,60.0
557752,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,JF,S49_A_01_6W_01,2.152915e-08,6.070312,60.0
557753,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,JF,S49_A_01_6W_01,2.152915e-08,5.080044,60.0
557754,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,JF,S49_A_01_6W_01,2.152915e-08,4.094509,60.0


In [54]:
pf_exceedance

,Receiver ID,Hazard ID,Consequence Type,Hazard Location,Frequency,Distance,Consequence Time
0,Pri Str 1W-1,S14_A_02_2P_01_100_ISBF,PF,S14_A_02_2P_01,7.552555e-07,21.963398,1.0
1,Pri Str 1W-1,S14_A_02_2P_01_100_IFBF,PF,S14_A_02_2P_01,4.899065e-08,21.963398,1.0
2,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,PF,S65_A_01_1W_12,3.375542e-08,11.034629,1.0
3,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,PF,S65_A_01_1W_12,3.375542e-08,10.917860,1.0
4,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,PF,S65_A_01_1W_12,3.375542e-08,10.849086,1.0
...,...,...,...,...,...,...,...
500482,Stool 8S-4,P09_A_02_6W_01_RUP_ISBF,PF,P09_A_02_6W_01,3.903072e-05,51.309859,2.0
500483,Stool 8S-4,P09_A_03_6W_01_RUP_ISBF,PF,P09_A_03_6W_01,2.609482e-05,51.309859,2.0
500484,Stool 8S-4,P09_A_01_6W_01_RUP_IFBF,PF,P09_A_01_6W_01,1.248057e-06,51.309859,60.0
500485,Stool 8S-4,P09_A_02_6W_01_RUP_IFBF,PF,P09_A_02_6W_01,2.531780e-06,51.309859,60.0


In [115]:
columns_template = ['Receiver ID JF','Hazard ID JF','Frequency JF','Consequence Time JF','Heat Dose JF','Cumulative Frequency JF','Receiver ID PF','Hazard ID PF','Frequency PF','Consequence Time PF','Heat Dose PF','Cumulative Frequency PF']
row_num = jf_exceedance.shape[0] if jf_exceedance.shape[0] > pf_exceedance.shape[0] else pf_exceedance.shape[0]
data = np.empty(shape = (row_num,len(columns_template)))
data[:] = np.nan
template_df = pd.DataFrame(data = data, columns = columns_template)

In [116]:
template_df.iloc[0:jf_exceedance.shape[0],0:4] = jf_exceedance[['Receiver ID','Hazard ID','Frequency','Consequence Time']]
template_df.iloc[0:pf_exceedance.shape[0],6:10] = pf_exceedance[['Receiver ID','Hazard ID','Frequency','Consequence Time']]
template_df.head()

,Receiver ID JF,Hazard ID JF,Frequency JF,Consequence Time JF,Heat Dose JF,Cumulative Frequency JF,Receiver ID PF,Hazard ID PF,Frequency PF,Consequence Time PF,Heat Dose PF,Cumulative Frequency PF
0,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,NaN,NaN,Pri Str 1W-1,S14_A_02_2P_01_100_ISBF,7.552555e-07,1.0,NaN,NaN
1,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,NaN,NaN,Pri Str 1W-1,S14_A_02_2P_01_100_IFBF,4.899065e-08,1.0,NaN,NaN
2,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,NaN,NaN,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,NaN,NaN
3,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,NaN,NaN,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,NaN,NaN
4,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,NaN,NaN,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,NaN,NaN


In [117]:
template_df["Heat Dose JF"] = 250 * template_df["Consequence Time JF"] * 60
template_df["Heat Dose PF"] = 100 * template_df["Consequence Time PF"] * 60
template_df.head()

,Receiver ID JF,Hazard ID JF,Frequency JF,Consequence Time JF,Heat Dose JF,Cumulative Frequency JF,Receiver ID PF,Hazard ID PF,Frequency PF,Consequence Time PF,Heat Dose PF,Cumulative Frequency PF
0,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,NaN,Pri Str 1W-1,S14_A_02_2P_01_100_ISBF,7.552555e-07,1.0,6000.0,NaN
1,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,NaN,Pri Str 1W-1,S14_A_02_2P_01_100_IFBF,4.899065e-08,1.0,6000.0,NaN
2,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,NaN,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,6000.0,NaN
3,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,NaN,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,6000.0,NaN
4,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,NaN,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,6000.0,NaN


In [118]:
template_df.iloc[0:jf_exceedance.shape[0]]["Cumulative Frequency JF"] = np.flip(np.cumsum(np.flip(template_df.iloc[0:jf_exceedance.shape[0]]["Frequency JF"])))
template_df.iloc[0:pf_exceedance.shape[0]]["Cumulative Frequency PF"] = np.flip(np.cumsum(np.flip(template_df.iloc[0:pf_exceedance.shape[0]]["Frequency PF"])))

C:\Users\pasindu.s\AppData\Local\Temp/ipykernel_28332/3487127733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_df.iloc[0:pf_exceedance.shape[0]]["Cumulative Frequency PF"] = np.flip(np.cumsum(np.flip(template_df.iloc[0:pf_exceedance.shape[0]]["Frequency PF"])))


In [119]:
template_df.head()

,Receiver ID JF,Hazard ID JF,Frequency JF,Consequence Time JF,Heat Dose JF,Cumulative Frequency JF,Receiver ID PF,Hazard ID PF,Frequency PF,Consequence Time PF,Heat Dose PF,Cumulative Frequency PF
0,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,0.049875,Pri Str 1W-1,S14_A_02_2P_01_100_ISBF,7.552555e-07,1.0,6000.0,0.317038
1,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,0.049875,Pri Str 1W-1,S14_A_02_2P_01_100_IFBF,4.899065e-08,1.0,6000.0,0.317038
2,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,0.049875,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,6000.0,0.317038
3,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,0.049875,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,6000.0,0.317038
4,Pri Str 1W-1,S15_A_03_1S_01_100_IFBF,1.451586e-09,0.0,0.0,0.049875,Pri Str 1W-1,S65_A_01_1W_12_050_ISBF,3.375542e-08,1.0,6000.0,0.317038


In [121]:
template_df.tail()

,Receiver ID JF,Hazard ID JF,Frequency JF,Consequence Time JF,Heat Dose JF,Cumulative Frequency JF,Receiver ID PF,Hazard ID PF,Frequency PF,Consequence Time PF,Heat Dose PF,Cumulative Frequency PF
557751,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,2.152915e-08,60.0,900000.0,1.076457e-07,NaN,NaN,NaN,NaN,NaN,NaN
557752,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,2.152915e-08,60.0,900000.0,8.611659e-08,NaN,NaN,NaN,NaN,NaN,NaN
557753,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,2.152915e-08,60.0,900000.0,6.458744e-08,NaN,NaN,NaN,NaN,NaN,NaN
557754,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,2.152915e-08,60.0,900000.0,4.305829e-08,NaN,NaN,NaN,NaN,NaN,NaN
557755,Pri Str 6W-12,S49_A_01_6W_01_050_IFBF,2.152915e-08,60.0,900000.0,2.152915e-08,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
total_rows = jf_exceedance.shape[0] + pf_exceedance.shape[0]
total_rows

1058243

In [124]:
columns_final = ["Receiver ID","Frequency","Consequence Time", "Heat Dose"]
data_final = np.empty(shape = (total_rows, len(columns_final)))
data_final[:] = np.nan
final_df = pd.DataFrame(data = data_final, columns = columns_final)
final_df

,Receiver ID,Frequency,Consequence Time,Heat Dose
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
1058238,NaN,NaN,NaN,NaN
1058239,NaN,NaN,NaN,NaN
1058240,NaN,NaN,NaN,NaN
1058241,NaN,NaN,NaN,NaN


In [135]:
final_df.iloc[0:jf_exceedance.shape[0],:] = template_df.iloc[0:jf_exceedance.shape[0]][["Receiver ID JF","Frequency JF","Consequence Time JF","Heat Dose JF"]]
final_df.iloc[jf_exceedance.shape[0]:final_df.shape[0],:] = template_df.iloc[0:pf_exceedance.shape[0]][["Receiver ID PF","Frequency PF","Consequence Time PF","Heat Dose PF"]]

In [136]:
final_df

,Receiver ID,Frequency,Consequence Time,Heat Dose
0,Pri Str 1W-1,1.451586e-09,0.0,0.0
28813,Pri Str 1W-1,1.451586e-09,0.0,0.0
28814,Pri Str 1W-1,1.451586e-09,0.0,0.0
28815,Pri Str 1W-1,1.451586e-09,0.0,0.0
28816,Pri Str 1W-1,1.451586e-09,0.0,0.0
...,...,...,...,...
325665,Stool 8S-4,3.903072e-05,2.0,12000.0
325664,Stool 8S-4,2.609482e-05,2.0,12000.0
325663,Stool 8S-4,1.248057e-06,60.0,360000.0
325675,Stool 8S-4,2.531780e-06,60.0,360000.0


In [137]:
final_df = final_df.sort_values(by = "Heat Dose", ignore_index = True)
final_df

,Receiver ID,Frequency,Consequence Time,Heat Dose
0,Pri Str 1W-1,1.451586e-09,0.0,0.0
1,Pri Str 1W-8,1.451586e-09,0.0,0.0
2,Pri Str 1W-8,1.451586e-09,0.0,0.0
3,Pri Str 1W-8,1.451586e-09,0.0,0.0
4,Pri Str 1W-8,1.451586e-09,0.0,0.0
...,...,...,...,...
1058238,Pri Str 3W-17,6.308353e-09,60.0,900000.0
1058239,Pri Str 3W-17,6.308353e-09,60.0,900000.0
1058240,Pri Str 3W-17,6.308353e-09,60.0,900000.0
1058241,Pri Str 3W-17,9.475549e-09,60.0,900000.0


In [138]:
final_df["Cumulative Frequency"] = np.nan
final_df["Cumulative Frequency"] = np.flip(np.cumsum(np.flip(final_df["Frequency"])))
final_df

,Receiver ID,Frequency,Consequence Time,Heat Dose,Cumulative Frequency
0,Pri Str 1W-1,1.451586e-09,0.0,0.0,3.669134e-01
1,Pri Str 1W-8,1.451586e-09,0.0,0.0,3.669134e-01
2,Pri Str 1W-8,1.451586e-09,0.0,0.0,3.669134e-01
3,Pri Str 1W-8,1.451586e-09,0.0,0.0,3.669134e-01
4,Pri Str 1W-8,1.451586e-09,0.0,0.0,3.669134e-01
...,...,...,...,...,...
1058238,Pri Str 3W-17,6.308353e-09,60.0,900000.0,3.655881e-08
1058239,Pri Str 3W-17,6.308353e-09,60.0,900000.0,3.025046e-08
1058240,Pri Str 3W-17,6.308353e-09,60.0,900000.0,2.394211e-08
1058241,Pri Str 3W-17,9.475549e-09,60.0,900000.0,1.763375e-08
